# 如何使用输出解析器将LLM响应解析为结构化格式

:::info 预备知识

本指南假定您熟悉以下概念：

- [输出解析器](/docs/concepts/output_parsers)
- [聊天模型](/docs/concepts/chat_models)

:::

语言模型输出的是文本。但在某些情况下，您可能希望获得比纯文本更结构化的信息。尽管某些模型提供商支持[返回结构化输出的内置方法](/docs/how_to/structured_output)，但并非全部都支持。对于这些提供商，您必须使用提示来引导模型以期望的格式返回结构化数据。

LangChain 提供了[输出解析器](/docs/concepts/output_parsers)，可帮助将模型输出解析为可用的对象。我们将在下面介绍几个示例。

## 开始之前

用于处理模型响应中结构化数据的主要输出解析器类型是 [`StructuredOutputParser`](https://api.js.langchain.com/classes/langchain_core.output_parsers.StructuredOutputParser.html)。在下面的示例中，我们将使用 [`zod`](https://zod.dev) 定义我们期望模型输出类型的模式：

首先，让我们看一下将插入提示中的默认格式说明：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs />
```

In [1]:
import { z } from "zod";
import { RunnableSequence } from "@langchain/core/runnables";
import { StructuredOutputParser } from "@langchain/core/output_parsers";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const zodSchema = z.object({
  answer: z.string().describe("answer to the user's question"),
  source: z.string().describe("source used to answer the user's question, should be a website."),
})

const parser = StructuredOutputParser.fromZodSchema(zodSchema);

const chain = RunnableSequence.from([
  ChatPromptTemplate.fromTemplate(
    "Answer the users question as best as possible.\n{format_instructions}\n{question}"
  ),
  model,
  parser,
]);

console.log(parser.getFormatInstructions());


You must format your output as a JSON value that adheres to a given "JSON Schema" instance.

"JSON Schema" is a declarative language that allows you to annotate and validate JSON documents.

For example, the example "JSON Schema" instance {{"properties": {{"foo": {{"description": "a list of test words", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}}}
would match an object with one required property, "foo". The "type" property specifies "foo" must be an "array", and the "description" property semantically describes it as "a list of test words". The items within "foo" must be strings.
Thus, the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of this example "JSON Schema". The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Your output will be parsed and type-checked according to the provided schema instance, so make sure all fields in your output match the schema exactly and there are no trailing commas!

Here is the JS

接下来，让我们调用这个链：

In [2]:
const response = await chain.invoke({
  question: "What is the capital of France?",
  format_instructions: parser.getFormatInstructions(),
});

console.log(response);

{
  answer: "The capital of France is Paris.",
  source: "https://en.wikipedia.org/wiki/Paris"
}


输出解析器实现了[Runnable接口](/docs/how_to/#langchain-expression-language-lcel)，这是[LangChain表达式语言（LCEL）](/docs/how_to/#langchain-expression-language-lcel)的基本构建块。这意味着它们支持`invoke`、`stream`、`batch`和`streamLog`调用。

## 验证

`StructuredOutputParser`的一个特性是它支持更严格的Zod验证。例如，如果你传递了一个不符合模式的模拟模型输出，我们将得到一个详细的类型错误：

In [3]:
import { AIMessage } from "@langchain/core/messages";

await parser.invoke(new AIMessage(`{"badfield": "foo"}`));

Error: Failed to parse. Text: "{"badfield": "foo"}". Error: [
  {
    "code": "invalid_type",
    "expected": "string",
    "received": "undefined",
    "path": [
      "answer"
    ],
    "message": "Required"
  },
  {
    "code": "invalid_type",
    "expected": "string",
    "received": "undefined",
    "path": [
      "source"
    ],
    "message": "Required"
  }
]

相较于：

In [4]:
await parser.invoke(new AIMessage(`{"answer": "Paris", "source": "I made it up"}`));

{ answer: "Paris", source: "I made it up" }

更高级的 Zod 验证也受支持。要了解更多信息，请查看 [Zod 文档](https://zod.dev)。

## 流式处理

虽然所有解析器都是可运行的并支持流式接口，但只有某些解析器能够通过部分解析的对象进行流式传输，因为这高度依赖于输出类型。`StructuredOutputParser` 不支持部分流式传输，因为它会在每个步骤验证输出。如果你尝试使用带有此输出解析器的链进行流式传输，该链将仅生成完全解析后的输出：

In [5]:
const stream = await chain.stream({
  question: "What is the capital of France?",
  format_instructions: parser.getFormatInstructions(),
});

for await (const s of stream) {
  console.log(s)
}

{
  answer: "The capital of France is Paris.",
  source: "https://en.wikipedia.org/wiki/Paris"
}


不过，更简单的 [`JsonOutputParser`](https://api.js.langchain.com/classes/langchain_core.output_parsers.JsonOutputParser.html) 支持通过部分输出进行流式传输：

In [8]:
import { JsonOutputParser } from "@langchain/core/output_parsers";

const template = `Return a JSON object with a single key named "answer" that answers the following question: {question}.
Do not wrap the JSON output in markdown blocks.`

const jsonPrompt = ChatPromptTemplate.fromTemplate(template);
const jsonParser = new JsonOutputParser();
const jsonChain = jsonPrompt.pipe(model).pipe(jsonParser);

const stream = await jsonChain.stream({
  question: "Who invented the microscope?",
});

for await (const s of stream) {
  console.log(s)
}

{}
{ answer: "" }
{ answer: "The" }
{ answer: "The invention" }
{ answer: "The invention of" }
{ answer: "The invention of the" }
{ answer: "The invention of the microscope" }
{ answer: "The invention of the microscope is" }
{ answer: "The invention of the microscope is attributed" }
{ answer: "The invention of the microscope is attributed to" }
{ answer: "The invention of the microscope is attributed to Hans" }
{ answer: "The invention of the microscope is attributed to Hans L" }
{
  answer: "The invention of the microscope is attributed to Hans Lippers"
}
{
  answer: "The invention of the microscope is attributed to Hans Lippershey"
}
{
  answer: "The invention of the microscope is attributed to Hans Lippershey,"
}
{
  answer: "The invention of the microscope is attributed to Hans Lippershey, Zach"
}
{
  answer: "The invention of the microscope is attributed to Hans Lippershey, Zacharias"
}
{
  answer: "The invention of the microscope is attributed to Hans Lippershey, Zacharias Jans"

## 下一步
你已经了解了如何使用输出解析器来解析模型输出中的结构化内容。

接下来，可以查看[工具调用指南](/docs/how_to/tool_calling)，这是某些模型提供商支持的一种更内置的获取结构化输出的方式；或者进一步阅读关于其他类型结构化数据（如[XML](/docs/how_to/output_parser_xml)）的输出解析器相关内容。